In [ ]:
!./build_and_push.sh yolov5

## Testing your algorithm on your local machine

When you're packaging your first algorithm to use with Amazon SageMaker, you probably want to test it yourself to make sure it's working correctly. We use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to test both locally and on SageMaker. For more examples with the SageMaker Python SDK, see [Amazon SageMaker Examples](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk). In order to test our algorithm, we need our dataset.

## SageMaker Python SDK Local Training
To represent our training, we use the Estimator class, which needs to be configured in five steps. 
1. IAM role - our AWS execution role
2. train_instance_count - number of instances to use for training.
3. train_instance_type - type of instance to use for training. For training locally, we specify `local` or `local_gpu`.
4. image_name - our custom PyTorch Docker image we created.
5. hyperparameters - hyperparameters we want to pass.

Let's start with setting up our IAM role. We make use of a helper function within the Python SDK. This function throw an exception if run outside of a SageMaker notebook instance, as it gets metadata from the notebook instance. If running outside, you must provide an IAM role with proper access stated above in [Permissions](#Permissions).

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

## Fit, Deploy, Predict

Now that the rest of our estimator is configured, we can call `fit()` with the path to our local CIFAR10 dataset prefixed with `file://`. This invokes our PyTorch container with 'train' and passes in our hyperparameters and other metadata as json files in /opt/ml/input/config within the container to our program entry point defined in the Dockerfile.

After our training has succeeded, our training algorithm outputs our trained model within the /opt/ml/model directory, which is used to handle predictions.

We can then call `deploy()` with an instance_count and instance_type, which is 1 and `local`. This invokes our PyTorch container with 'serve', which setups our container to handle prediction requests as defined [here](https://github.com/aws/sagemaker-pytorch-container/blob/master/src/sagemaker_pytorch_container/serving.py#L103). What is returned is a predictor, which is used to make inferences against our trained model.

After our prediction, we can delete our endpoint.

We recommend testing and training your training algorithm locally first, as it provides quicker iterations and better debuggability.

In [ ]:
# # Lets set up our SageMaker notebook instance for local mode.
# !/bin/bash ./utils/setup.sh

In [ ]:
# import os
# import subprocess

# instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
# print("Instance type = " + instance_type)

In [ ]:
# from sagemaker.estimator import Estimator

# hyperparameters = {}

# estimator = Estimator(role=role,
#                       train_instance_count=1,
#                       train_instance_type=instance_type,
#                       image_uri='yolov5:latest',
#                       hyperparameters=hyperparameters)

# estimator.fit({'cfg': 'file:///home/ec2-user/SageMaker/yolov5_sagemaker/data/cfg/', 'weights': 'file:///home/ec2-user/SageMaker/yolov5_sagemaker/data/weights/', 'images': 'file:///home/ec2-user/SageMaker/yolov5_sagemaker/data/images/', 'labels': 'file:///home/ec2-user/SageMaker/yolov5_sagemaker/data/labels/'})


In [ ]:
# TODO
#predictor = estimator.deploy(1, instance_type)

## Making predictions using Python SDK

To make predictions, we will use a few images, from the test loader, converted into a json format to send as an inference request.

The reponse will be tensors containing the probabilities of each image belonging to one of the 10 classes. Based on the highest probability we will map that index to the corresponding class in our output. The classes can be referenced from the [CIFAR-10 website](https://www.cs.toronto.edu/~kriz/cifar.html). Since we didn't train the model for that long, we aren't expecting very accurate results.

In [ ]:
# # TODO
# import torchvision, torch
# import numpy as np

# from sagemaker.predictor import json_serializer, json_deserializer

# # get some test images
# dataiter = iter(testloader)
# images, labels = dataiter.next()

# # print images
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%4s' % classes[labels[j]] for j in range(4)))

# predictor.accept = 'application/json'
# predictor.content_type = 'application/json'

# predictor.serializer = json_serializer
# predictor.deserializer = json_deserializer

# outputs = predictor.predict(images.numpy())

# _, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

# print('Predicted: ', ' '.join('%4s' % classes[predicted[j]]
#                               for j in range(4)))

In [ ]:
# predictor.delete_endpoint()

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker
Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment
Here we specify the bucket to use and the role that is used for working with SageMaker.

In [ ]:
# S3 prefix
prefix = 'DEMO-pytorch-yolov5'

## Create the session

The session remembers our connection parameters to SageMaker. We use it to perform all of our SageMaker operations.

In [ ]:
import sagemaker as sage

sess = sage.Session()

## Upload the data for training

We will use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [ ]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/yolov5_sagemaker/data/'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

inputs = {'cfg': data_location+'/cfg', 'weights': data_location+'/weights', 'images': data_location+'/images', 'labels': data_location+'/labels'}

print(inputs)

## Training on SageMaker
Training a model on SageMaker with the Python SDK is done in a way that is similar to the way we trained it locally. This is done by changing our train_instance_type from `local` to one of our [supported EC2 instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/).

In addition, we must now specify the ECR image URL, which we just pushed above.

Finally, our local training dataset has to be in Amazon S3 and the S3 URL to our dataset is passed into the `fit()` call.

Let's first fetch our ECR image url that corresponds to the image we just built and pushed.

In [ ]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'yolov5'

if region.startswith('cn'):
    ecr_image = '{}.dkr.ecr.{}.amazonaws.com.cn/{}:latest'.format(account, region, algorithm_name)
else:
    ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {}

instance_type = 'ml.p3.2xlarge' #  'ml.p3.2xlarge'

estimator = Estimator(role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(inputs)

In [ ]:
instance_type = 'ml.m5.xlarge'
predictor = estimator.deploy(1, instance_type)

## Optional cleanup
When you're done with the endpoint, you should clean it up.

All of the training jobs, models and endpoints we created can be viewed through the SageMaker console of your AWS account.

In [ ]:
predictor.delete_endpoint()

# Reference
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)
- [SageMaker PyTorch container](https://github.com/aws/sagemaker-pytorch-container)